In [1]:
import os
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
current_dir = os.getcwd()
dir_warehouse = f"{current_dir}/warehouse"

In [3]:
spark = SparkSession.builder \
    .appName("IcebergWithSpark") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", dir_warehouse) \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .getOrCreate()

25/01/06 21:10:17 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface wlp0s20f3)
25/01/06 21:10:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/06 21:10:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Exclui a tabela se existir
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas")

DataFrame[]

In [5]:
# Cria a tabela Vendas no catalogo, usando Iceberg
spark.sql("""
CREATE TABLE hadoop_catalog.default.vendas (
    id INT,
    produto STRING,
    quantidade INT,
    preco DOUBLE,
    data_venda DATE
)
USING iceberg
""")

DataFrame[]

In [6]:
# Incluindo dados na tabela vendas
data = [
    (1, "Produto A", 10, 15.5, "2024-11-01"),
    (2, "Produto B", 5, 22.0, "2024-11-02"),
    (3, "Produto C", 8, 30.0, "2024-11-03")
]
columns = ["id", "produto", "quantidade", "preco", "data_venda"]
df = spark.createDataFrame(data, columns)
df = df.withColumn("data_venda", F.to_date(F.col("data_venda"), "yyyy-MM-dd"))

In [7]:
# Gravamos os dados na tabela vendas
df.writeTo("hadoop_catalog.default.vendas").append()

In [9]:
# Verificamos os dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2024-11-01|
|  2|Produto B|         5| 22.0|2024-11-02|
|  3|Produto C|         8| 30.0|2024-11-03|
+---+---------+----------+-----+----------+



In [10]:
# Atualizamos o preço do produto de id 1
spark.sql("""
UPDATE hadoop_catalog.default.vendas
SET preco = 17.0
WHERE id = 1
""")

DataFrame[]

In [11]:
# Verificamos a atualização
spark.sql("SELECT * FROM hadoop_catalog.default.vendas WHERE id = 1").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 17.0|2024-11-01|
+---+---------+----------+-----+----------+



In [12]:
# Excluimos o registro de número 2
spark.sql("""
DELETE FROM hadoop_catalog.default.vendas
WHERE id = 2
""")

DataFrame[]

In [13]:
# verificamos a exclusão
spark.sql("SELECT * FROM hadoop_catalog.default.vendas").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  3|Produto C|         8| 30.0|2024-11-03|
|  1|Produto A|        10| 17.0|2024-11-01|
+---+---------+----------+-----+----------+



In [14]:
# excluimos a tabela do catalogo
spark.sql("DROP TABLE hadoop_catalog.default.vendas")

DataFrame[]